In [1]:
!pip install langchain langchain-community langchain-openai tiktoken --quiet

In [10]:
from aimon import AnalyzeEval, Application, Model

In [11]:
import os
openai_api_key = os.getenv("OPENAI_API_KEY")

In [12]:
aimon_config = {"hallucination": {"detector_name": "default"},
                "conciseness": {"detector_name": "default"},
                "completeness": {"detector_name": "default"},
                "toxicity": {"detector_name": "default"},
                "instruction_adherence": {"detector_name": "default"}
                }

### Create a dataset and a dataset collection

AIMon can be used to manage datasets used for the evaluations. Here, we create two datasets and a collection
comprising of these datasets. Note that these datasets only need to be created once.

In [13]:
import json
# Create a new datasets
file_path1 = "./test_evaluation_dataset_aug_2024_1.csv"
file_path2 = "./test_evaluation_dataset_aug_2024_2.csv"

dataset_data_1 = json.dumps({
    "name": "test_evaluation_dataset_aug_2024_first.csv",
    "description": "This is one custom dataset"
})

dataset_data_2 = json.dumps({
    "name": "test_evaluation_dataset_aug_2024_second.csv",
    "description": "This is another custom dataset"
})

from aimon import Client
aimon_api_key = os.getenv("AIMON_API_KEY")
aimon_client = Client(auth_header=f"Bearer {aimon_api_key}")

with open(file_path1, 'rb') as file1:
    dataset1 = aimon_client.datasets.create(
        file=file1,
        json_data=dataset_data_1
    )

with open(file_path2, 'rb') as file2:
    dataset2 = aimon_client.datasets.create(
        file=file2,
        json_data=dataset_data_2
    )
dataset1 = aimon_client.datasets.list(name="test_evaluation_dataset_aug_2024_first.csv")
dataset2 = aimon_client.datasets.list(name="test_evaluation_dataset_aug_2024_second.csv")

In [14]:
# Create a new dataset collection
dataset_collection = aimon_client.datasets.collection.create(
    name="my_first_dataset_collection_aug_9_2024", 
    dataset_ids=[dataset1.sha, dataset2.sha], 
    description="This is a collection of two datasets."
)

### Evaluation

We will run an evaluation the example below that uses Langchain to summarize documents using OpenAI.

In [19]:
analyze_eval = AnalyzeEval(
    Application("llm_marketing_summarization_app_v3"),
    Model("my_gpt4_model_fine_tuned", "GPT-4"), 
    headers=['context_docs', 'user_query', 'prompt', 'instructions'],
    api_key=os.getenv("AIMON_API_KEY"),
    evaluation_name="simple_eval",
    dataset_collection_name="my_first_dataset_collection_aug_9_2024",
    config=aimon_config
)

In [20]:
# Lanchain app example
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.llms.openai import OpenAI
from langchain.chains.summarize import load_summarize_chain

# eval = Eval(aimon_config)
# eval.dataset_collection(data_coll_name)

# The analyze_eval decorator will automatically stream through
# records in the specified data collection and run it against 
# this function. The signature of this function should necessarily 
# contain context_docs, user_query and prompt as the first 3 
# arguments. If you plan to provide "instructions" to compute the
# instruction_adherence metric, make sure there is a 4th parameter 
# to this decorated function called "instructions".
@analyze_eval
def run_application_eval_mode(context_docs=None, user_query=None, prompt=None, instructions=None, my_arg=None):
    # Split the source text
    text_splitter = CharacterTextSplitter()
    texts = text_splitter.split_text(context_docs)
    
    # Create Document objects for the texts
    docs = [Document(page_content=t) for t in texts[:3]]
    
    # Initialize the OpenAI module, load and run the summarize chain
    llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    return chain.run(docs) 

In [21]:
aimon_eval_res = run_application_eval_mode()

In [22]:
print(aimon_eval_res)

[(' Acme recently launched version 2.1 of their Python library, which has deep integrations with the Python ecosystem and has been proven to be valuable for developers. This new version includes features like async support and improved error handling. Acme also supports Javascript and Java.', AnalyzeCreateResponse(message='Data successfully sent to AIMon.', status=200)), ('\n\nTo configure the Acme python client, follow the official documentation which includes setting up environment variables and installing dependencies for both basic and advanced setups.', AnalyzeCreateResponse(message='Data successfully sent to AIMon.', status=200)), (' The Acme python client is compatible with Python 3.6+ and multiple databases, including MySQL, PostgreSQL, and MongoDB. It is also suitable for cross-language projects with Node.js.', AnalyzeCreateResponse(message='Data successfully sent to AIMon.', status=200)), (' The Acme python client may have installation, package conflicts, and connectivity iss